# Lab. 4: (Reduced) Variable Neighborhood Search

## Introduction

#### <u>The goal of this laboratory is to compare the behavior of VNS and RVNS on the knapsack 0/1 problem.</u>

You'll have to implement two methods:

- shake(x, k). Try different implementation of the shake method:
  - flip k bits taken randomly in the binary vector x
  - flip max(1, k-1) bits in the binary vector x
  - flip at least k bits in the binary vector x
  - ...
- local_search(f, x)
  - different implementations of VNS use as local search either FirstImprovement or BestImprovement algorithms

---

Getting started: The following cells contain the implementation of the methods that we will use throughout this lab, together with utilities.

**The Knapsack 0/1 problem** is a combinatorial problem that works as follows.

We have a bag with limited capacity L, and we have a set of items  $I=\{i_1,...,i_N\}$  Each  $i_j$  has a given volume $w_{ij}$ and a value $v_{ij}$. The goal of the optimization problem is to fill the bag with a combination of items $S$ such that the combination maximizes the total value contained by the bag while complaining with the constraint on the capacity.

In this version of the knapsack problem we can either discard or carry (at most 1) item. Thus, the search space is $\{0,1\}^{|I|}$ .

Since this is a maximization problem, we can turn it into a minimization problem by returning the opposite of the value. If a solution is not feasible (i.e., exceeds the maximum volume) the function will return a score of 0 (like an empty bag).


Assume
$$ N = \text{number of items} $$
$$ X = \{x_1, x_2, \dots, x_N\} = \text{set of items} $$
$$ v_i = \text{value of item } x_i $$
$$ w_i = \text{weight of item } x_i $$
$$ W = \text{maximum weight the knapsack can hold} $$


Then the 0/1 knapsack problem can be formulated as follows:

\begin{equation*}
  \begin{array}{ll}
    \text{maximize }   &S = \sum_{x_i \in X} v_i x_i \\
    \text{subject to } & \sum_{x_i \in X} w_i x_i \leq W \\
                       & x_i \in \{0, 1\} \forall x_i \in X \\
  \end{array}
\end{equation*}

In [ ]:
import random
from copy import deepcopy
from dataclasses import dataclass
from itertools import combinations
from pprint import pformat, pprint
from typing import Literal

import matplotlib.pyplot as plt

random.seed(42)


@dataclass(frozen=True)
class Item:
    name: str
    value: int
    volume: int

    def __str__(self) -> str:
        return pformat({"name": self.name, "value": self.value, "volume": self.volume})


class Knapsack_0_1:
    def __init__(self) -> None:
        self.items = [
            Item("apple", 1, 2),
            Item("pear", 2, 2),
            Item("banana", 2, 2),
            Item("watermelon", 5, 10),
            Item("orange", 3, 2),
            Item("avocado", 3, 2),
            Item("blueberry", 3, 1),
            Item("coconut", 4, 3),
            Item("cherry", 2, 1),
            Item("apricot", 1, 1),
        ]
        self.BAG_CAPACITY = 10
        self.history: list[list[int]] = []
        self.values: list[int] = []

    def _get_value(self, solution: list[int]) -> int:
        cur_cap = self.BAG_CAPACITY
        cur_val = 0
        for i, v in enumerate(solution):
            if v == 1:
                cur_val += self.items[i].value
                cur_cap -= self.items[i].volume
            if cur_cap < 0:
                return 0
        return -cur_val

    def trend(self) -> None:
        plt.figure()
        plt.plot(self.values)
        plt.show()

    def info(self, items_mask: list[int]) -> None:
        items = [self.items[i] for i in range(len(items_mask)) if items_mask[i] == 1]
        capacity = sum([i.volume for i in items])
        quantity = sum([i.value for i in items])
        pprint(
            {
                "items": items,
                "max capacity": self.BAG_CAPACITY,
                "achived capacity": capacity,
                "quantity": quantity,
            },
            sort_dicts=False,
        )

    def __call__(self, solution: list[int]):
        value = self._get_value(solution)
        self.history.append(solution)
        self.values.append(value)
        return value

In [ ]:
def shake(x: list[int], k: int) -> list[int]:
    """
    Implements the shaking step for the VNS algorithm. It generates a random x' contained in the neighborhood k of x

    - x: starting point from which the neighborhood is created
    - k: the neighborhood distance (1 for the nearest, 2 for a slightly bigger neighborhood, and so on)

    Returns:
    - The random x' sampled from the neighborhood
    """

    # create a copy of x
    xp = deepcopy(x)

    # flip k random bits of xs
    indexes = random.sample(range(len(xp)), k)
    for i in indexes:
        xp[i] = 1 - xp[i]
    return xp


def local_search(
    f: Knapsack_0_1, x: list[int], k: int, type: Literal["first"] | Literal["best"]
) -> list[int]:
    """
    Implements the local search step for the VNS algorithm. It search for a better x' in the neighborhood

    - f: function to minimize
    - x: starting point of the local search

    Returns:
    - The improved x'
    """
    xb = deepcopy(x)

    # generate all possible combinations that flip k bits
    combs = list(combinations(range(len(x)), k))
    if type == "first":
        # perform a first-improvement local search
        for indexes in combs:
            xs = deepcopy(x)
            for i in indexes:
                xs[i] = 1 - xs[i]
            if f(xs) < f(xb):
                xb = xs
                break
    elif type == "best":
        # perform a best-improvement local search
        for indexes in combs:
            xs = deepcopy(x)
            for i in indexes:
                xs[i] = 1 - xs[i]
            if f(xs) < f(xb):
                xb = xs
    else:
        raise ValueError("type must be 'first' or 'best'")

    return xb


def move_or_not(
    f: Knapsack_0_1, x: list[int], xs: list[int], k: int
) -> tuple[list[int], int]:
    if f(xs) < f(x):
        x = xs
        k = 1
    else:
        k += 1
    return x, k


def vns(
    f: Knapsack_0_1,
    x0: list[int],
    kmax: int,
    ls_method: Literal["first"] | Literal["best"],
) -> list[int]:
    """
    Seeks for the minimum of the function by means of the Variable Neighborhood
    Search algorithm.

    :f: the function to optimize
    :x0: the initial point
    :kmax: the max number of iterations for each neighborhood
    """
    x = deepcopy(x0)
    best = x
    bestval = f(x)

    k = 0
    while k < kmax:
        xp = shake(x, k)
        xs = local_search(f, xp, k, type=ls_method)
        x, k = move_or_not(f, x, xs, k)

        # Store the best
        curval = f(x)
        if curval < bestval:
            bestval = curval
            best = x

    obtained_capacity = sum(
        [f.items[i].volume for i in range(len(best)) if best[i] == 1]
    )
    if obtained_capacity > f.BAG_CAPACITY:
        best = [0] * len(best)
    return best


def rvns(f: Knapsack_0_1, x0: list[int], kmax: int) -> list[int]:
    """
    Seeks for the minimum of the function by means of the Reduced Variable
    Neighborhood Search algorithm.

    :f: the function to optimize
    :x0: the initial point
    :kmax: the max number of iterations for each neighborhood
    """
    x = deepcopy(x0)
    best = x
    bestval = f(x)

    k = 0
    while k < kmax:
        xp = shake(x, k)
        x, k = move_or_not(f, x, xp, k)

        # Store the best
        curval = f(x)
        if curval < bestval:
            bestval = curval
            best = x

    obtained_capacity = sum(
        [f.items[i].volume for i in range(len(best)) if best[i] == 1]
    )
    if obtained_capacity > f.BAG_CAPACITY:
        print("overcapacity ", obtained_capacity)
        best = [0] * len(best)
    return best

# Variable Neighborhood Search
---
## Questions:
- how does the starting point influence the search process?
- how does the kmax parameter affect the quality of the result?
- how does the generation of the neighborhood (for the shaking) affect:
    - quality of the search?
    - speed of the search?

In [ ]:
func = Knapsack_0_1()
max_neighboorhood = [
    [1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
for starting_point in max_neighboorhood:
    best = vns(func, starting_point, 10, "first")
    print("Start point: ", starting_point)
    print("Final point: ", best)
    func.info(best)
    func.trend()

In [ ]:
func = Knapsack_0_1()
starting_point = [1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
max_neighboorhood = [2, 5, 7, 10]
for k in max_neighboorhood:
    best = vns(func, starting_point, k, "first")
    print("Start point:", starting_point)
    print("Final point:", best)
    print("Neighborhood size:", k)
    func.info(best)
    func.trend()

In [ ]:
func = Knapsack_0_1()
starting_point = [1, 0, 0, 1, 0, 1, 1, 0, 0, 0]
best = vns(func, starting_point, 10, "first")
func.info(best)
func.trend()

In [ ]:
func = Knapsack_0_1()
starting_point = [1, 0, 0, 1, 0, 1, 1, 0, 0, 0]
best = vns(func, starting_point, 10, "best")
func.info(best)
func.trend()

# Reduced Variable Neighborhood Search
---
## Questions:
- how does the starting point influence the search process?
- how does the kmax parameter affect the quality of the result?
- how does the generation of the neighborhood affect:
    - quality of the search?
    - velocity of the search?
- how does RVNS compare to VNS?

In [ ]:
func = Knapsack_0_1()
starting_point = [1, 0, 0, 1, 0, 1, 1, 0, 0, 0]
best = rvns(func, starting_point, 10)
func.info(best)
func.trend()

In [ ]:
func = Knapsack_0_1()
max_neighboorhood = [
    [1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
for starting_point in max_neighboorhood:
    best = rvns(func, starting_point, 10)
    print("Start point: ", starting_point)
    print("Final point: ", best)
    func.info(best)

In [ ]:
func = Knapsack_0_1()
starting_point = [1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
max_neighboorhood = [2, 5, 7, 10]
for k in max_neighboorhood:
    best = rvns(func, starting_point, k)
    print("Start point:", starting_point)
    print("Final point:", best)
    print("Neighborhood size:", k)
    func.info(best)